In [1]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
import sys
sys.path.append("../../")
from src.config import Configuration

config = Configuration()

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Server RAG

In [2]:
from src.rag.hybrid_gemini import HybridGeminiRag
INDEX = "labse-major"
MODEL = "sentence-transformers/LaBSE"
rag = HybridGeminiRag(config=config, es_index=INDEX, embed_model=MODEL)


From (44, 4) to Counter({2: 37, 1: 6, 3: 1})
From (44, 4) to 44


In [ ]:
a = rag.chain.invoke()

## Local Follow Up

In [3]:
from src.prepare.data_load import DocDataLoader

doc_loader = DocDataLoader()
docs = doc_loader.load_major_docs_full()
d1 = docs[15]
d1.page_content[:30]

From (44, 4) to 44


'Kỹ thuật hoá học là ngành học'

In [4]:
from src.chain.followup import LocalFollowUpChain
followup = LocalFollowUpChain(config=config, doc_content=d1.page_content)

## Router

In [6]:
from langchain_core.runnables import RunnableBranch


branch = RunnableBranch(
    (lambda x: "local" in x["topic"].lower(), followup.chain),
    rag.chain
)


In [8]:
llm = config.get_gemini_pro(convert_system_message=False)
template = """Given the user question below, classify it as either being about `Local`, or `Other`. Known that:
```
    Local: when the question related to "ngành kỹ thuật hóa học" or "ngành kỹ thuật hóa học trường Tôn Đức Thắng".
```
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
chain = (
    PromptTemplate.from_template(template=template)
    | llm
    | StrOutputParser()
)

In [9]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [10]:
config.enable_tracing(project="LEARN")

In [11]:
from langsmith.run_helpers import traceable

@traceable(tags=['route'])
def run_full_chain(q):
    a = full_chain.invoke({"question": q})
    return a

In [13]:
q = "em muốn anh tư vấn ngành kỹ thuật hóa học ạ"
a = run_full_chain(q)
a

AIMessage(content='**Điểm mạnh của ngành Kỹ thuật Hóa học tại Trường Đại học Tôn Đức Thắng:**\n\n* Chương trình đào tạo quốc tế\n* Trang bị kiến thức chuyên sâu về hóa lý, vật liệu vô cơ, vật liệu hữu cơ, tách chiết hợp chất\n* Thực hành chiếm 50% chương trình đào tạo với các phòng thí nghiệm hiện đại\n* Kết nối doanh nghiệp chặt chẽ, tạo cơ hội trải nghiệm thực tế và thực hiện dự án nghiên cứu\n* Hỗ trợ sinh viên tham gia các dự án nghiên cứu khoa học và cuộc thi học thuật\n\n**Cơ hội nghề nghiệp:**\n\n* Chuyên viên vận hành và quản lý sản xuất\n* Chuyên viên nghiên cứu phát triển sản phẩm\n* Chuyên viên tư vấn quản lý và chuyển giao công nghệ\n* Nhân viên kinh doanh hóa chất\n* Nhân viên kiểm soát chất lượng sản phẩm\n* Nghiên cứu, giảng dạy tại các trung tâm, viện nghiên cứu, trường đại học cao đẳng')